In [ ]:
from_drive=True

In [ ]:
## GIT ON COLAB ONLY
try:
    from google.colab import userdata
    git_token = userdata.get('gitToken')
    git_user = userdata.get('gitUser')
    git_url = f'https://{git_token}@github.com/rene-aum/Atlas.git'
    branch_to_pull = 'dev'
    !git clone {git_url}
    %cd Atlas
    !git checkout -b colab_branch
    !git pull origin {branch_to_pull}
    !pip install -r PipelinesConsumo/src/requirements.txt
    %cd PipelinesConsumo
except Exception as e:
    print(e)
    print('Running in other environment not colab probably!')

In [ ]:
import sys
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import pytz
# from matplotlib.ticker import FuncFormatter
from datetime import datetime, timedelta
import warnings
import sys
sys.path.append('..')
sys.path.append('../..')
from utils.utils import (get_dates_dataframe,
                       add_year_week,
                       custom_read,
                       process_columns)
from src.loader import Loader
from src.transformed import Transformed
from utils.drive_toolbox import(from_drive_to_local,
                             get_last_modification_date_drive,
                             create_file_in_drive_folder,
                             update_file_in_drive_folder,
                             read_from_google_sheets)
from src.constants import (
                           id_tracking_envios,
                           id_reporte_ventas,
                           id_business_case,
                           id_master_table_sheets,
                           id_clientes_apartado_sheets,
                           id_resultados_squad,
                           id_business_case_processed,
                           id_medios_total_sheets
                           )


warnings.filterwarnings('ignore')



In [ ]:
if from_drive:
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    import gspread
    from google.auth import default
    from gspread_dataframe import set_with_dataframe
    import gdown
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    ######################################################### get ids per file ###############################################################
    folder_id = '1hl7hITBmAXZKkb_qAHZ-045yJnSwfU2Q'

    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
    file_id_dict = {}
    for file in file_list:
        file_id_dict[file['title']] = file['id']
    id_data_warehouse_drive = file_id_dict['Data Warehouse - Productivo.xlsx']
    id_CTA_adobe_csv = file_id_dict['CTAAdobe.csv']
    id_unique_visitors_adobe = file_id_dict['UniqueVisitorsAdobe.csv']
    id_medios_total_csv = file_id_dict['medios_total.csv']
    id_cancelaciones = file_id_dict['VS_cancellation_log.csv']
    id_channel_appstep = file_id_dict['ChannelAppStep.csv']
    id_product_views = file_id_dict['pdp-hist.csv']
    ##################################################### get all files to local filesystem ###################################################
    from_drive_to_local(drive,id_data_warehouse_drive,'dw_automarket.xlsx')
    from_drive_to_local(drive,id_tracking_envios,'tracking_campañas_automarket_datio.xlsx')
    from_drive_to_local(drive,id_unique_visitors_adobe,'unique_visitors_adobe.csv')
    from_drive_to_local(drive,id_CTA_adobe_csv,'CTAAdobe.csv')
    from_drive_to_local(drive,id_medios_total_csv, 'medios_total.csv')
    from_drive_to_local(drive,id_cancelaciones,'cancelaciones.csv')
    from_drive_to_local(drive,id_channel_appstep,'ChannelAppStep.csv')
    from_drive_to_local(drive,id_product_views,'pdp-hist.csv')
    ###########################################################################################################################################
    data_date = get_last_modification_date_drive(drive,id_data_warehouse_drive)
    ###########################################################################################################################################
    creds, _ = default()
    gc = gspread.authorize(creds)

In [ ]:
lo = Loader()

In [ ]:
path_excel = 'dw_automarket.xlsx'#'/Users/mi36597/Documents/automarket/dw_productivo_raw/dw_productivo_20250213.xlsx'
path_envios = 'tracking_campañas_automarket_datio.xlsx'
path_visitors_adobe = 'unique_visitors_adobe.csv'#"/Users/mi36597/Documents/automarket/UniqueVisitorsAdobe.csv"
path_salud_inventario = 'salud_de_inventario_completo.csv'
path_CTA_adobe = 'CTAAdobe.csv'
path_fuentes = 'fuentes.csv'
path_fuentes_funnel = 'fuentes_funnel.csv'
path_channel_appstep = 'ChannelAppStep.csv'
path_pdp = 'pdp-hist.csv'

## Reportes 

In [ ]:
all_dates = get_dates_dataframe()
try:
    envios_df = lo.load_envios_medios_propios(excel_path = path_envios, excel_tab_name = 'total')
except Exception as e:
    print(e)
reporte_ventas = (read_from_google_sheets(gc, id_reporte_ventas))
business_case_df = read_from_google_sheets(gc,id_business_case,sheetname='Looker')
pedidos_logs = lo.load_pedidos_tab(excel_path=path_excel,excel_tab_name='Pedidos')
new_users = lo.load_clientes_tab(excel_path=path_excel,excel_tab_name='Clientes')
veh_status = lo.load_vehicle_status_tab(excel_path=path_excel,excel_tab_name='Vehicle Status')
appstep_df = lo.load_appstep_tab(csv_path=path_channel_appstep)
p_views = lo.load_product_views_tab(csv_path=path_pdp)
vs_url = pd.read_excel(path_excel,sheet_name='Vehicle Status URL')


In [ ]:
tr = Transformed(lo)

In [ ]:
url_mod = (vs_url
 .pipe(process_columns)
 .rename(columns = {'vs_extra_url_key':'url',
                    'customer_id':'id_am',
                    'niv/vin':'vin'
                    })
 [lambda x: x.url.notna()]
 .assign(published_at = lambda x: pd.to_datetime(x.published_at).dt.strftime('%Y-%m-%d'))
[['sku','vin','published_at','status','status_product','plate','product_name','product_price','showroom','url']]
 )

In [ ]:
# clientes
clientes = pd.read_excel(path_excel,sheet_name='Clientes')

# clientes.columns
customer_df = (clientes
              .pipe(process_columns)
              [['id','email','phone','customer_since','phone_number_otp_validated','email_otp_validated' ]]
              .assign(phone = lambda x: x.phone.astype('Int64'),
                      customer_since = lambda x: pd.to_datetime(x.customer_since).dt.strftime('%Y-%m-%d') ,
                      phone_number_otp_validated = lambda x: x.phone_number_otp_validated.fillna(0),
                      email_otp_validated = lambda x: x.email_otp_validated.fillna(0))
              [lambda x: x.phone_number_otp_validated.eq(1)|x.email_otp_validated.eq(1)]
              .rename(columns={'id':'id_am'})
              .assign(dwh_date=data_date)
              )
customer_to_match = customer_df[['id_am','email','phone','customer_since','dwh_date']]
customer_censored = (customer_df[['id_am','customer_since','dwh_date']].copy())
# publicaciones
veh_status.columns
veh_status.columns
publicaciones = (veh_status
                [['id_am','product_id','published_date','status','status_product',
                  'product_price','sku','vin','plate','product_name','showroom']]
                 .assign(flag_publicacion_activa = lambda x: np.where((x.status=='Enabled')&
                                                                      (x.status_product.isin(['published','reserved'])),
                                                                      1,0),
                         dwh_date=data_date)
                )
# apartados
status_apartados = ['Entrega de auto (finalizado)',
                    'Pedido cancelado por negociación no exitosa por parte del comprador',
                    'Pedido cancelado por no-show del comprador', 
                    'Auto en cita',
                    'Revisión de auto',
                    'Acuerdo de compraventa',
                    'Negociación de precio']
pedidos_intermedia = (tr.t_pedidos_intermedia()
                      .assign(dwh_date=data_date,
                              fecha_de_creacion = lambda x: x.fecha_de_creacion.dt.strftime('%Y-%m-%d')
                              
                              )
                      )

### Export to drive

In [ ]:
id_cliente_sheets = '1pOsV7nRFkYhyRv4wLaNay_kWoJ_9IBjVlZ860lCAHQU'
id_publicaciones_sheets = '1bPFOiYGSIjZRJteWH_gwRndVatHQAikru9pFhGCY4m4'
id_detalle_pedido = '1j4iAfY7uJrtOMk1Yi6rxYSe7Z80Fh3lm-Nkj6ytghYQ'
id_usuarios_para_match = '1EtrjRSDrVa44vFSjpbbP6sHEDdWVjFsh-9QbWX8ff5k'
id_vehicle_url = '15gQfqzxS15Uw4aGJFeZ0M9nvg_XQJlR4rqsFhXJ8YU8'

In [ ]:
update_file_in_drive_folder(gc,id_cliente_sheets,'Hoja 1',customer_censored)
update_file_in_drive_folder(gc,id_publicaciones_sheets,'Hoja 1',publicaciones)
update_file_in_drive_folder(gc,id_detalle_pedido,'Hoja 1',pedidos_intermedia)
update_file_in_drive_folder(gc,id_usuarios_para_match,'Hoja 1',customer_to_match)
update_file_in_drive_folder(gc,id_vehicle_url,'Hoja 1',url_mod)

## Tabla user activities

In [ ]:
act_users = (new_users
             [['id_am', 'customer_since']]

             )
### summary publicaciones ######
summary_publicaciones = (veh_status
                         [lambda x: x.published_date.notna()]
                         .sort_values(by='published_date', ascending=False)
                         .groupby('id_am', as_index=False)
                         .agg(n_publicaciones=('sku', 'nunique'),
                              first_published_date=('published_date', 'min'),
                              last_published_date=('published_date', 'max'),
                              last_published_vin=('vin', lambda x: x.head(1)),
                              last_published_product_name=(
                             'product_name', lambda x: x.tail(1))
                         )
                         )
### pdp ult 30 días ######
pdp_l30d = (p_views
            [lambda x: x.id_am.notna()
             & (x.id_am != 'undefined')
                & (pd.to_datetime(x.date) >= (datetime.now()-timedelta(days=30)))
             ]
            .assign(nsku=lambda x: x.groupby('id_am').sku.transform('nunique'))
            .sort_values(by='date', ascending=False)
            .groupby('id_am', as_index=False)
            .agg({'nsku': 'max',
                  'url': lambda x: x.head(1),
                  'date': lambda x: x.head(1)})
            .assign(last_pdp_car_description=lambda x: x.url.str.split('/').str[-1].str.split('.html').str[0],
                    id_am=lambda x: (pd.to_numeric(x.id_am,errors='coerce')).astype('Int64'))
            [lambda x: x.id_am.notna()]
            .rename(columns={'nsku': 'nsku_pdp_l30d',
                             'url': 'last_pdp_url',
                             'date': 'last_pdp_date'})

            )
### summary inicios de funnel######
summary_inicios_id = (appstep_df
                      [lambda x: x.id_am.notna()
                       & (x.application_name.isin(['vender mi auto', 'pago de apartado']))
                          & x.app_page_visit.gt(0)

                       ]
                      .assign(date=lambda x: x.date.dt.strftime('%Y-%m-%d'))
                      .groupby(['id_am', 'application_name'])
                      .agg(first_inicio_funnel=('date', 'min'),
                           n_inicio_funnel=('app_page_visit',
                                            lambda x: (x > 0).sum()),
                           last_incio_funnel=('date', 'max')
                           )
                      .unstack('application_name')

                      )
summary_inicios_id.columns = ['first_inicio_apartado', 'first_inicio_publicacion', 'n_inicio_apartado_unique',
                              'n_inicio_publicacion_unique', 'last_inicio_apartado', 'last_inicio_de_publicacion']
summary_inicios_id = summary_inicios_id.reset_index()[['id_am', 'first_inicio_apartado', 'last_inicio_apartado',
                                                       'n_inicio_apartado_unique', 'first_inicio_publicacion',
                                                         'last_inicio_de_publicacion', 'n_inicio_publicacion_unique']]

### primeros apartados ######
primeros_apartados_id = (pedidos_intermedia
                         .sort_values(by=['id_am_comprador', 'fecha_de_creacion'], ascending=[True, True])
                         .assign(n_apartados=lambda x: x.groupby('id_am_comprador').numero_de_pedido.transform('nunique'),
                                 n_multiapartados=lambda x: x.groupby('id_am_comprador').multiapartado.transform('sum'))
                         .groupby('id_am_comprador').head(1)
                         [['id_am_comprador', 'fecha_de_creacion',
                           'numero_de_pedido', 'n_apartados', 'n_multiapartados']]
                         .rename(columns={'fecha_de_creacion': 'first_apartado_date',
                                          'id_am_comprador': 'id_am',
                                          'numero_de_pedido': 'first_numero_de_pedido'
                                          })
                         .reset_index(drop=True)
                         )
### ultimos apartados ######
ultimos_apartados_id = (pedidos_intermedia
                        .sort_values(by=['id_am_comprador', 'fecha_de_creacion'], ascending=[True, True])
                        .groupby('id_am_comprador').tail(1)
                        [['id_am_comprador', 'fecha_de_creacion', 'numero_de_pedido']]
                        .rename(columns={'fecha_de_creacion': 'last_apartado_date',
                                         'id_am_comprador': 'id_am',
                                         'numero_de_pedido': 'last_numero_de_pedido'
                                         })
                        .reset_index(drop=True)
                        )
### ventas ######
vin_comprador = (pedidos_intermedia
                 .assign(vin=lambda x: x.vin.str.upper().str.strip())
                 .sort_values(by=['vin', 'fecha_de_creacion'], ascending=[True, False])
                 .groupby('vin').head(1)
                 [['vin', 'id_am_comprador', 'numero_de_pedido']]
                 .rename(columns={'id_am_comprador': 'id_am'})
                 .reset_index(drop=True)
                 )

reporte_ventas_aux = (reporte_ventas
                      .rename(columns={'Año': 'anio'})
                      .pipe(process_columns)
                      .assign(fecha_de_entrega=lambda x: pd.to_datetime(x.fecha_de_entrega, format='%d/%m/%Y').dt.strftime('%Y-%m-%d'),
                              vin=lambda x: x.vin.str.upper().str.strip(),
                              numero_de_pedido=lambda x: x.id_pedido.astype(
                                  'Int64')
                              )
                      )


ventas_id = (reporte_ventas_aux
             .merge(vin_comprador[['id_am', 'numero_de_pedido']], on='numero_de_pedido', how='left')
             .sort_values(by=['id_am', 'fecha_de_entrega'], ascending=[True, False])
             .merge(vin_comprador[['id_am', 'vin']], on='vin', how='left')
             .assign(id_am=lambda x: x.id_am_y.fillna(x.id_am_x),
                     last_venta_descripcion=lambda x: x['marca'].astype(
                 str)+'-'+x['modelo'].astype(str)+'-'+(x['anio'].fillna(-1).astype(int).astype(str)),
             )
             .assign(n_sales=lambda x: x.groupby('id_am').vin.transform('nunique'),
                     id_am=lambda x: x.id_am.astype('Int64'))
             .assign(n_sales=lambda x: x.n_sales.fillna(0))
             .drop(columns=['id_am_x', 'id_am_y', 'marca', 'modelo', 'anio'])
             .sort_values(by=['id_am', 'fecha_de_entrega'], ascending=[True, False])
             [['id_am', 'fecha_de_entrega', 'vin', 'numero_de_pedido',
               'n_sales', 'last_venta_descripcion']]
             .groupby('id_am').head(1)
             .rename(columns={'vin': 'last_sale_vin',
                              'fecha_de_entrega': 'last_sale_date',
                              'numero_de_pedido': 'last_sale_numero_de_pedido'})
             .reset_index(drop=True)
             )

### apps completed ######
apps_completed_df_vender = (appstep_df
                     [lambda x: x.id_am.notna()
                      & (x.application_name.isin(['vender mi auto']))
                         & x.app_step_10.gt(0)

                      ]
                     .assign(date=lambda x: x.date.dt.strftime('%Y-%m-%d'))
                     .groupby(['id_am', 'application_name'])
                     .agg(first_apps_completed=('date', 'min'),
                          n_apps_completed=('app_step_10',
                                            lambda x: (x > 0).sum()),
                          last_apps_completed=('date', 'max')
                          )
                     .unstack('application_name')

                     )
apps_completed_df_vender.columns = [x[0]+' '+x[1] for x in apps_completed_df_vender]
apps_completed_df_vender = (apps_completed_df_vender
                     .rename(columns={'first_apps_completed pago de apartado': 'first_app_completed_apartado',
                                      'first_apps_completed vender mi auto': 'first_app_completed_vender',
                                      'n_apps_completed pago de apartado': 'n_app_completed_apartado',
                                      'n_apps_completed vender mi auto': 'n_app_completed_vender',
                                      'last_apps_completed pago de apartado': 'last_completed_apartado',
                                      'last_apps_completed vender mi auto': 'last_app_completed_vender'})
                     )

apps_completed_df_comprar = (appstep_df
                     [lambda x: x.id_am.notna()
                      & (x.application_name.isin(['pago de apartado']))
                         & x.app_completed.gt(0)

                      ]
                     .assign(date=lambda x: x.date.dt.strftime('%Y-%m-%d'))
                     .groupby(['id_am', 'application_name'])
                     .agg(first_apps_completed=('date', 'min'),
                          n_apps_completed=('app_completed',
                                            lambda x: (x > 0).sum()),
                          last_apps_completed=('date', 'max')
                          )
                     .unstack('application_name')

                     )
apps_completed_df_comprar.columns = [x[0]+' '+x[1] for x in apps_completed_df_comprar]
apps_completed_df_comprar = (apps_completed_df_comprar
                     .rename(columns={'first_apps_completed pago de apartado': 'first_app_completed_apartado',
                                      
                                      'n_apps_completed pago de apartado': 'n_app_completed_apartado',
                                      
                                      'last_apps_completed pago de apartado': 'last_completed_apartado'})
                     )



###  app step 4 vendedor ####

aps4_vendedor_df = (appstep_df
                    [lambda x: x.id_am.notna()
                     & (x.application_name.isin(['vender mi auto']))
                        & x.app_step_4.gt(0)

                     ]
                    .assign(date=lambda x: x.date.dt.strftime('%Y-%m-%d'))
                    .groupby(['id_am', 'application_name'])
                    .agg(first_aps4_vender=('date', 'min'),
                         n_aps4_vender=('app_step_4',
                                        lambda x: (x > 0).sum()),
                         last_aps4_vender=('date', 'max')
                         )
                    .reset_index()
)

In [ ]:
#### user activities table #########
fl_table = (act_users
            .merge(summary_inicios_id, on='id_am', how='left')
            .merge(primeros_apartados_id, on='id_am', how='left')
            .merge(ultimos_apartados_id, on='id_am', how='left')
            .merge(ventas_id, on='id_am', how='left')
            .merge(pdp_l30d, on='id_am', how='left')
            .merge(summary_publicaciones, on='id_am', how='left')
            .merge(apps_completed_df_vender, on='id_am', how='left')
            .merge(apps_completed_df_comprar, on='id_am', how='left')
            .merge(aps4_vendedor_df, on='id_am', how='left')
            )
cols_fillna_zero = [x for x in fl_table.dtypes[lambda x: x.isin(['Int64', 'float64'])].index.values.tolist(
) if x not in ['last_numero_de_pedido', 'first_numero_de_pedido', 'last_numero_de_pedido']]
fl_table[cols_fillna_zero] = fl_table[cols_fillna_zero].fillna(0)

## order columns ###


ordered_final_columns = ['id_am', 'customer_since',
                         'first_inicio_apartado', 'last_inicio_apartado', 'n_inicio_apartado_unique',
                         'first_app_completed_apartado', 'n_app_completed_apartado', 'last_completed_apartado',
                         'first_apartado_date', 'first_numero_de_pedido', 'n_apartados', 'n_multiapartados',
                         'last_apartado_date', 'last_numero_de_pedido',
                         'last_sale_date', 'last_sale_vin', 'last_sale_numero_de_pedido', 'n_sales', 'last_venta_descripcion',
                         'first_inicio_publicacion', 'last_inicio_de_publicacion', 'n_inicio_publicacion_unique',
                         'first_aps4_vender', 'n_aps4_vender', 'last_aps4_vender',
                         'first_app_completed_vender', 'n_app_completed_vender', 'last_app_completed_vender',
                         'first_published_date', 'last_published_date', 'last_published_vin', 'last_published_product_name', 'n_publicaciones',
                         'nsku_pdp_l30d', 'last_pdp_url','last_pdp_date','last_pdp_car_description']

In [ ]:
final_user_activities = fl_table[ordered_final_columns].copy()
id_user_activities = "1nhXfN-ehUb51leb7e1SCrm6hRd9TadKAaWStjb-gIQU"
update_file_in_drive_folder(gc,id_user_activities,'Hoja 1',final_user_activities)